
SAR Image Colorization — Pix2Pix / U-Net (PyTorch)
This is a Jupyter-friendly .py notebook (cells separated by `# %%`).
It implements a paired image-to-image translation training pipeline for
SAR -> RGB colorization using Lab supervision (predict a/b channels).
Features:
- PyTorch implementation (generator = U-Net, discriminator = PatchGAN)
- Paired dataset loader for SAR (grayscale) and RGB images
- Optional SAR contrast stretch (percentile clipping + dB conversion)
- Data augmentations (paired random crop, flips, rotations)
- Losses: LSGAN adversarial loss + L1 on a,b channels
- AMP (automatic mixed precision), checkpointing, TensorBoard logging
- Inference utilities to save predicted RGB outputs

Edit configuration at the CONFIG cell and run.

CONFIG


In [12]:
import os
from pathlib import Path

# Path to the root dataset folder. The dataset should contain pairs like:
# <dataset_root>/<terrain>/s1/<name>.png  (SAR grayscale)
# <dataset_root>/<terrain>/s2/<name>.png  (RGB)
DATA_ROOT = Path(r"D:\My Disk\coding\v_2")  # change to your dataset path

# Training hyperparameters
DEVICE = 'cuda' if (os.environ.get('CUDA_VISIBLE_DEVICES') or True) and __import__('torch').cuda.is_available() else 'cpu'
BATCH_SIZE = 8
PATCH_SIZE = 256
NUM_EPOCHS = 50
LR = 2e-4
BETA1 = 0.5
BETA2 = 0.999
WEIGHT_DECAY = 1e-4
LAMBDA_L1 = 100.0
CHECKPOINT_DIR = Path('./checkpoints')
LOG_DIR = Path('./logs')
NUM_WORKERS = 0

# Data splitting
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

# SAR preprocessing options
USE_SAR_DB = True
SAR_PERCENTILE_CLIP = True
CLIP_LO = 2.0
CLIP_HI = 98.0

# Misc
SAVE_SAMPLE_EVERY = 1  # epochs
AMP_ENABLED = True
PRINT_EVERY_BATCHES = 50

CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
print(DEVICE)

cuda


Imports

In [4]:

import random
import math
import time
import numpy as np
from PIL import Image
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.utils as vutils
from torch.utils.tensorboard import SummaryWriter
from torch.amp import GradScaler
from skimage import color

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Utilities: Lab conversions & SAR preprocessing


In [5]:
EPS = 1e-8

def rgb_to_lab_tensor(rgb_uint8: np.ndarray):
    """rgb_uint8: HxWx3 uint8 image (0-255) -> returns L (0-100), a (-128,127), b (-128,127) as float32 arrays"""
    rgb = rgb_uint8.astype('float32') / 255.0
    lab = color.rgb2lab(rgb)
    return lab.astype('float32')


def lab_to_rgb_uint8(lab: np.ndarray):
    """lab: HxWx3 float32 (L 0-100, a,b around -128..127) -> uint8 RGB 0-255"""
    rgb = color.lab2rgb(lab)
    rgb = np.clip(rgb * 255.0, 0, 255).astype('uint8')
    return rgb


def sar_preprocess(img_pil: Image.Image, use_db=USE_SAR_DB, do_clip=SAR_PERCENTILE_CLIP, lo_pct=CLIP_LO, hi_pct=CLIP_HI):
    """Take a PIL grayscale SAR image -> return normalized L channel in range [0,1]
    Steps:
      - convert to float32
      - scale to [0,1]
      - optionally apply dB: 20*log10(img + eps)
      - optionally percentile clip and rescale
    """
    arr = np.array(img_pil).astype('float32')
    # If image is 3-ch grayscale repeated, reduce
    if arr.ndim == 3:
        arr = arr[..., 0]
    # normalize naive 0-255 -> 0-1
    arr = arr / 255.0
    if use_db:
        # Convert amplitude-like to dB
        arr = 20.0 * np.log10(np.clip(arr, EPS, None))
    if do_clip:
        lo = np.percentile(arr, lo_pct)
        hi = np.percentile(arr, hi_pct)
        arr = np.clip(arr, lo, hi)
        if hi - lo > 0:
            arr = (arr - lo) / (hi - lo)
        else:
            arr = np.clip(arr, 0, 1)
    # final clamp
    arr = np.clip(arr, 0.0, 1.0)
    return arr.astype('float32')

Paired Dataset

In [6]:
class PairedSARDataset(Dataset):
    def __init__(self, root: Path, terrains=None, split='train', patch_size=PATCH_SIZE,
                 use_db=USE_SAR_DB, clip_sar=SAR_PERCENTILE_CLIP, augment=True):
        """
        root: dataset root
        terrains: list of terrain subfolders to include (None => all)
        split: 'train'|'val'|'test'
        """
        self.root = Path(root)
        all_pairs = []
        terrains = terrains or [p.name for p in self.root.iterdir() if p.is_dir()]
        for t in terrains:
            s1_dir = self.root / t / 's1'
            s2_dir = self.root / t / 's2'
            if not s1_dir.exists() or not s2_dir.exists():
                continue
            # find image names common to both
            s1_files = sorted([p for p in s1_dir.glob('*.png')])
            for p in s1_files:
                name = p.name
                s2p = s2_dir / name.replace('_s1_', '_s2_') if '_s1_' in name else (s2_dir / name)
                # fallback: same basename
                if not s2p.exists():
                    s2p = s2_dir / name
                if s2p.exists():
                    all_pairs.append((str(p), str(s2p)))
        # shuffle and split
        random.shuffle(all_pairs)
        n = len(all_pairs)
        ntest = int(n * TEST_SPLIT)
        nval = int(n * VAL_SPLIT)
        if split == 'test':
            self.pairs = all_pairs[:ntest]
        elif split == 'val':
            self.pairs = all_pairs[ntest:ntest + nval]
        else:
            self.pairs = all_pairs[ntest + nval:]

        self.patch_size = patch_size
        self.augment = augment
        self.use_db = use_db
        self.clip_sar = clip_sar

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        s1_path, s2_path = self.pairs[idx]
        s1_pil = Image.open(s1_path).convert('L')  # SAR grayscale
        s2_pil = Image.open(s2_path).convert('RGB')  # RGB

        # optionally random crop to patch size
        if self.augment:
            w, h = s1_pil.size
            if w >= self.patch_size and h >= self.patch_size:
                x = random.randint(0, w - self.patch_size)
                y = random.randint(0, h - self.patch_size)
                s1_pil = s1_pil.crop((x, y, x + self.patch_size, y + self.patch_size))
                s2_pil = s2_pil.crop((x, y, x + self.patch_size, y + self.patch_size))
            else:
                # center crop or resize to patch size
                s1_pil = s1_pil.resize((self.patch_size, self.patch_size), Image.BILINEAR)
                s2_pil = s2_pil.resize((self.patch_size, self.patch_size), Image.BILINEAR)

            # random flips and 90-degree rotations
            if random.random() > 0.5:
                s1_pil = s1_pil.transpose(Image.FLIP_LEFT_RIGHT)
                s2_pil = s2_pil.transpose(Image.FLIP_LEFT_RIGHT)
            if random.random() > 0.5:
                s1_pil = s1_pil.transpose(Image.FLIP_TOP_BOTTOM)
                s2_pil = s2_pil.transpose(Image.FLIP_TOP_BOTTOM)
            if random.random() > 0.5:
                s1_pil = s1_pil.transpose(Image.ROTATE_90)
                s2_pil = s2_pil.transpose(Image.ROTATE_90)

        else:
            s1_pil = s1_pil.resize((self.patch_size, self.patch_size), Image.BILINEAR)
            s2_pil = s2_pil.resize((self.patch_size, self.patch_size), Image.BILINEAR)

        # Preprocess SAR -> L channel normalized to [0,1]
        L = sar_preprocess(s1_pil, use_db=self.use_db, do_clip=self.clip_sar)
        # RGB -> Lab
        rgb = np.array(s2_pil)
        lab = rgb_to_lab_tensor(rgb)
        # L_ref from RGB may not match SAR L; we will use SAR-derived L as the luminance guide.
        # Extract a,b channels and scale to [-1,1]
        ab = lab[..., 1:3]
        ab = (ab.astype('float32')) / 128.0  # approx scale to [-1,1]

        # Convert to tensors
        # SAR L (1,H,W)
        L_t = torch.from_numpy(L).unsqueeze(0)
        # ab (2,H,W)
        ab_t = torch.from_numpy(ab).permute(2, 0, 1)
        # For visualization/metrics we also return the reference rgb and lab L
        rgb_t = torch.from_numpy(rgb.astype('float32') / 255.0).permute(2, 0, 1)
        labL_ref = torch.from_numpy(lab[..., :1].astype('float32') / 100.0).permute(2, 0, 1)

        return {
            'sar_L': L_t,       # [1,H,W], normalized [0,1]
            'ab': ab_t,         # [2,H,W], approx [-1,1]
            'rgb': rgb_t,       # [3,H,W], [0,1]
            'labL_ref': labL_ref  # [1,H,W], [0,1]
        }


Model definitions: UNet generator and PatchGAN discriminator

In [7]:
# A small conv block

def conv_block(in_ch, out_ch, norm=True):
    layers = [nn.Conv2d(in_ch, out_ch, kernel_size=4, stride=2, padding=1, bias=False)]
    if norm:
        layers.append(nn.InstanceNorm2d(out_ch, affine=True))
    layers.append(nn.LeakyReLU(0.2, inplace=True))
    return nn.Sequential(*layers)


class UNetGenerator(nn.Module):
    def __init__(self, in_channels=1, out_channels=2, ngf=64, dropout=True):
        super().__init__()
        # encoder
        self.enc1 = nn.Sequential(nn.Conv2d(in_channels, ngf, 4, 2, 1), nn.LeakyReLU(0.2, inplace=True))
        self.enc2 = conv_block(ngf, ngf * 2)
        self.enc3 = conv_block(ngf * 2, ngf * 4)
        self.enc4 = conv_block(ngf * 4, ngf * 8)
        self.enc5 = conv_block(ngf * 8, ngf * 8)
        self.enc6 = conv_block(ngf * 8, ngf * 8)
        self.enc7 = conv_block(ngf * 8, ngf * 8)
        self.enc8 = conv_block(ngf * 8, ngf * 8, norm=False)

        # decoder (transpose conv)
        self.dec1 = nn.Sequential(nn.ConvTranspose2d(ngf * 8, ngf * 8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 8), nn.ReLU(True))
        self.dec2 = nn.Sequential(nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 8), nn.ReLU(True))
        self.dec3 = nn.Sequential(nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 8), nn.ReLU(True))
        self.dec4 = nn.Sequential(nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 8), nn.ReLU(True))
        self.dec5 = nn.Sequential(nn.ConvTranspose2d(ngf * 16, ngf * 4, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 4), nn.ReLU(True))
        self.dec6 = nn.Sequential(nn.ConvTranspose2d(ngf * 8, ngf * 2, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf * 2), nn.ReLU(True))
        self.dec7 = nn.Sequential(nn.ConvTranspose2d(ngf * 4, ngf, 4, 2, 1, bias=False),
                                   nn.InstanceNorm2d(ngf), nn.ReLU(True))
        self.dec8 = nn.Sequential(nn.ConvTranspose2d(ngf * 2, out_channels, 4, 2, 1), nn.Tanh())

        # dropout in deeper dec layers
        self.dropout = nn.Dropout(0.5) if dropout else nn.Identity()

    def forward(self, x):
        # encoder
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        e5 = self.enc5(e4)
        e6 = self.enc6(e5)
        e7 = self.enc7(e6)
        e8 = self.enc8(e7)

        d1 = self.dec1(e8)
        d1 = torch.cat([d1, e7], dim=1)
        d1 = self.dropout(d1)
        d2 = self.dec2(d1)
        d2 = torch.cat([d2, e6], dim=1)
        d3 = self.dec3(d2)
        d3 = torch.cat([d3, e5], dim=1)
        d4 = self.dec4(d3)
        d4 = torch.cat([d4, e4], dim=1)
        d5 = self.dec5(d4)
        d5 = torch.cat([d5, e3], dim=1)
        d6 = self.dec6(d5)
        d6 = torch.cat([d6, e2], dim=1)
        d7 = self.dec7(d6)
        d7 = torch.cat([d7, e1], dim=1)
        out = self.dec8(d7)
        return out


# PatchGAN Discriminator
class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=3, ndf=64):
        # in_channels: number of channels of input (L + ab) -> we will feed SAR L concatenated with predicted/real ab => 1+2 =3
        super().__init__()
        layers = []
        layers.append(nn.utils.spectral_norm(nn.Conv2d(in_channels, ndf, 4, 2, 1)))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        layers.append(nn.utils.spectral_norm(nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False)))
        layers.append(nn.InstanceNorm2d(ndf * 2, affine=True))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        layers.append(nn.utils.spectral_norm(nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False)))
        layers.append(nn.InstanceNorm2d(ndf * 4, affine=True))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        layers.append(nn.utils.spectral_norm(nn.Conv2d(ndf * 4, ndf * 8, 4, 1, 1, bias=False)))
        layers.append(nn.InstanceNorm2d(ndf * 8, affine=True))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        layers.append(nn.utils.spectral_norm(nn.Conv2d(ndf * 8, 1, 4, 1, 1)))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


Training loop helpers

In [8]:
def save_checkpoint(state, filename: Path):
    torch.save(state, str(filename))


def load_checkpoint(filename: Path, device):
    return torch.load(str(filename), map_location=device)

Training

In [9]:
def train_loop(dataset_root: Path):
    # dataset and loaders
    train_ds = PairedSARDataset(dataset_root, split='train', patch_size=PATCH_SIZE, augment=True)
    val_ds = PairedSARDataset(dataset_root, split='val', patch_size=PATCH_SIZE, augment=False)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

    # models
    netG = UNetGenerator(in_channels=1, out_channels=2).to(DEVICE)
    netD = PatchDiscriminator(in_channels=3).to(DEVICE)

    # losses
    criterion_GAN = nn.MSELoss()  # LSGAN
    criterion_L1 = nn.L1Loss()

    # optimizers
    optG = torch.optim.Adam(netG.parameters(), lr=LR, betas=(BETA1, BETA2), weight_decay=WEIGHT_DECAY)
    optD = torch.optim.Adam(netD.parameters(), lr=LR, betas=(BETA1, BETA2), weight_decay=WEIGHT_DECAY)

    scaler = torch.amp.GradScaler(device=DEVICE, enabled=AMP_ENABLED)

    writer = SummaryWriter(log_dir=str(LOG_DIR))

    best_val_l1 = float('inf')

    global_step = 0
    for epoch in range(1, NUM_EPOCHS + 1):
        netG.train(); netD.train()
        running_loss_G = 0.0
        running_loss_D = 0.0
        running_l1 = 0.0
        t0 = time.time()
        for i, batch in enumerate(train_loader):
            sar_L = batch['sar_L'].to(DEVICE)  # [B,1,H,W]
            real_ab = batch['ab'].to(DEVICE)   # [B,2,H,W]

            # Prepare real and fake labels for LSGAN
            valid = torch.ones((sar_L.size(0), 1, 30, 30), device=DEVICE)  # size depends on patch size; discriminator will handle it
            fake = torch.zeros_like(valid)

            # ------------------
            # Train Discriminator
            # ------------------
            with torch.autocast(device_type=DEVICE,enabled=AMP_ENABLED):
                fake_ab = netG(sar_L)
                # concatenate L + ab -> 3 channels
                real_input_D = torch.cat([sar_L, real_ab], dim=1)
                fake_input_D = torch.cat([sar_L, fake_ab.detach()], dim=1)
                pred_real = netD(real_input_D)
                pred_fake = netD(fake_input_D)
                loss_D_real = criterion_GAN(pred_real, valid)
                loss_D_fake = criterion_GAN(pred_fake, fake)
                loss_D = (loss_D_real + loss_D_fake) * 0.5

            optD.zero_grad()
            scaler.scale(loss_D).backward()
            scaler.step(optD)

            # ------------------
            # Train Generator
            # ------------------
            with torch.autocast(device_type=DEVICE,enabled=AMP_ENABLED):
                fake_ab = netG(sar_L)
                fake_input_D = torch.cat([sar_L, fake_ab], dim=1)
                pred_fake = netD(fake_input_D)
                loss_G_GAN = criterion_GAN(pred_fake, valid)
                loss_G_L1 = criterion_L1(fake_ab, real_ab) * LAMBDA_L1
                loss_G = loss_G_GAN + loss_G_L1

            optG.zero_grad()
            scaler.scale(loss_G).backward()
            scaler.step(optG)
            scaler.update()

            running_loss_G += loss_G.item()
            running_loss_D += loss_D.item()
            running_l1 += loss_G_L1.item()

            if (i + 1) % PRINT_EVERY_BATCHES == 0:
                print(f"Epoch [{epoch}/{NUM_EPOCHS}] Batch [{i+1}/{len(train_loader)}] "
                      f"LossG: {running_loss_G/(i+1):.4f} LossD: {running_loss_D/(i+1):.4f} L1: {running_l1/(i+1):.4f}")

            global_step += 1

        t1 = time.time()
        print(f"Epoch {epoch} finished in {t1-t0:.1f}s — LossG: {running_loss_G/len(train_loader):.4f} LossD: {running_loss_D/len(train_loader):.4f}")

        # Validation
        netG.eval(); netD.eval()
        val_l1 = 0.0
        with torch.no_grad():
            for vb, vbatch in enumerate(val_loader):
                sar_L = vbatch['sar_L'].to(DEVICE)
                real_ab = vbatch['ab'].to(DEVICE)
                fake_ab = netG(sar_L)
                val_l1 += criterion_L1(fake_ab, real_ab).item()
        val_l1 = val_l1 / len(val_loader)
        writer.add_scalar('val/L1', val_l1, epoch)

        # Save checkpoint
        ckpt_path = CHECKPOINT_DIR / f'ckpt_epoch_{epoch}.pth'
        save_checkpoint({
            'epoch': epoch,
            'netG_state': netG.state_dict(),
            'netD_state': netD.state_dict(),
            'optG': optG.state_dict(),
            'optD': optD.state_dict(),
            'val_l1': val_l1
        }, ckpt_path)

        # Save best
        if val_l1 < best_val_l1:
            best_val_l1 = val_l1
            save_checkpoint({
                'epoch': epoch,
                'netG_state': netG.state_dict(),
                'netD_state': netD.state_dict(),
                'optG': optG.state_dict(),
                'optD': optD.state_dict(),
                'val_l1': val_l1
            }, CHECKPOINT_DIR / 'best.pth')

        # Save sample images (reconstructed RGB)
        if epoch % SAVE_SAMPLE_EVERY == 0:
            sample_dir = Path('./samples')
            sample_dir.mkdir(parents=True, exist_ok=True)
            # take first batch from val loader
            vbatch = next(iter(val_loader))
            sar_L = vbatch['sar_L'].to(DEVICE)
            real_rgb = vbatch['rgb']
            with torch.no_grad():
                fake_ab = netG(sar_L).cpu().numpy()  # [B,2,H,W]
            # reconstruct rgb from SAR-derived L + predicted ab
            sar_L_cpu = vbatch['sar_L'].cpu().numpy()  # [B,1,H,W]
            B = sar_L_cpu.shape[0]
            for bi in range(min(B, 8)):
                L = (sar_L_cpu[bi, 0] * 100.0).astype('float32')  # [H,W]
                ab = (fake_ab[bi].transpose(1, 2, 0) * 128.0).astype('float32')
                lab = np.concatenate([L[..., None], ab], axis=2)
                rgb_uint8 = lab_to_rgb_uint8(lab)
                out_p = sample_dir / f'epoch{epoch}_sample{bi}.png'
                Image.fromarray(rgb_uint8).save(out_p)

    writer.close()

 Inference util: load best model and run on arbitrary SAR image, save RGB

In [10]:
def inference_on_image(sar_img_path: str, out_path: str, checkpoint_path: Path = CHECKPOINT_DIR / 'best.pth', use_db=USE_SAR_DB, clip_sar=SAR_PERCENTILE_CLIP):
    device = DEVICE
    ckpt = load_checkpoint(checkpoint_path, device)
    netG = UNetGenerator(in_channels=1, out_channels=2).to(device)
    netG.load_state_dict(ckpt['netG_state'])
    netG.eval()

    img = Image.open(sar_img_path).convert('L')
    # optionally resize to multiples of 256 or keep original; UNet is flexible but expects sizes divisible by 2^8
    w, h = img.size
    # ensure sizes divisible by 256? We will pad to nearest multiple of 256
    multiple = 256
    pad_w = (multiple - (w % multiple)) % multiple
    pad_h = (multiple - (h % multiple)) % multiple
    if pad_w or pad_h:
        new_w = w + pad_w
        new_h = h + pad_h
        img = img.resize((new_w, new_h), Image.BILINEAR)

    L = sar_preprocess(img, use_db=use_db, do_clip=clip_sar)
    L_t = torch.from_numpy(L).unsqueeze(0).unsqueeze(0).to(device)  # [1,1,H,W]

    with torch.no_grad():
        fake_ab = netG(L_t)
    fake_ab = fake_ab.cpu().numpy()[0].transpose(1, 2, 0)  # H,W,2
    L_np = (L * 100.0).astype('float32')
    lab = np.concatenate([L_np[..., None], (fake_ab * 128.0).astype('float32')], axis=2)
    rgb_uint8 = lab_to_rgb_uint8(lab)
    Image.fromarray(rgb_uint8).save(out_path)


In [13]:
if __name__ == '__main__':
    # Set multiprocessing start method (important for Windows)
    import multiprocessing
    multiprocessing.set_start_method('spawn', force=True)
    
    # Run the training loop
    train_loop(DATA_ROOT)

Epoch [1/50] Batch [50/1600] LossG: 32.5488 LossD: 0.5623 L1: 30.6623
Epoch [1/50] Batch [100/1600] LossG: 23.0459 LossD: 0.3310 L1: 21.5861
Epoch [1/50] Batch [150/1600] LossG: 19.1125 LossD: 0.2412 L1: 17.7842
Epoch [1/50] Batch [200/1600] LossG: 16.8688 LossD: 0.1959 L1: 15.6348
Epoch [1/50] Batch [250/1600] LossG: 15.4417 LossD: 0.1810 L1: 14.2544
Epoch [1/50] Batch [300/1600] LossG: 14.4004 LossD: 0.1654 L1: 13.2556
Epoch [1/50] Batch [350/1600] LossG: 13.5528 LossD: 0.1491 L1: 12.4363
Epoch [1/50] Batch [400/1600] LossG: 12.9516 LossD: 0.1378 L1: 11.8564
Epoch [1/50] Batch [450/1600] LossG: 12.4913 LossD: 0.1279 L1: 11.4114
Epoch [1/50] Batch [500/1600] LossG: 12.0982 LossD: 0.1188 L1: 11.0300
Epoch [1/50] Batch [550/1600] LossG: 11.7612 LossD: 0.1124 L1: 10.7021
Epoch [1/50] Batch [600/1600] LossG: 11.4696 LossD: 0.1069 L1: 10.4176
Epoch [1/50] Batch [650/1600] LossG: 11.2392 LossD: 0.1029 L1: 10.1922
Epoch [1/50] Batch [700/1600] LossG: 11.0199 LossD: 0.1000 L1: 9.9806
Epoch [1

C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 10 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)
C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 35 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)
C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 25 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)
C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 15 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)
C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ t

Epoch [2/50] Batch [50/1600] LossG: 7.9507 LossD: 0.0140 L1: 6.9608
Epoch [2/50] Batch [100/1600] LossG: 7.9316 LossD: 0.0142 L1: 6.9308
Epoch [2/50] Batch [150/1600] LossG: 7.9736 LossD: 0.0120 L1: 6.9731
Epoch [2/50] Batch [200/1600] LossG: 8.0614 LossD: 0.0108 L1: 7.0603
Epoch [2/50] Batch [250/1600] LossG: 8.0578 LossD: 0.0100 L1: 7.0567
Epoch [2/50] Batch [300/1600] LossG: 8.0906 LossD: 0.0096 L1: 7.0892
Epoch [2/50] Batch [350/1600] LossG: 8.0510 LossD: 0.0097 L1: 7.0496
Epoch [2/50] Batch [400/1600] LossG: 8.0484 LossD: 0.0110 L1: 7.0467
Epoch [2/50] Batch [450/1600] LossG: 8.0350 LossD: 0.0109 L1: 7.0319
Epoch [2/50] Batch [500/1600] LossG: 8.0056 LossD: 0.0105 L1: 7.0030
Epoch [2/50] Batch [550/1600] LossG: 7.9934 LossD: 0.0100 L1: 6.9904
Epoch [2/50] Batch [600/1600] LossG: 8.0123 LossD: 0.0100 L1: 7.0088
Epoch [2/50] Batch [650/1600] LossG: 8.0171 LossD: 0.0101 L1: 7.0134
Epoch [2/50] Batch [700/1600] LossG: 7.9772 LossD: 0.0099 L1: 6.9734
Epoch [2/50] Batch [750/1600] LossG

C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 4 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)


Epoch [4/50] Batch [50/1600] LossG: 7.9369 LossD: 0.0023 L1: 6.9329
Epoch [4/50] Batch [100/1600] LossG: 8.0159 LossD: 0.0015 L1: 7.0140
Epoch [4/50] Batch [150/1600] LossG: 7.9009 LossD: 0.0013 L1: 6.8992
Epoch [4/50] Batch [200/1600] LossG: 7.8672 LossD: 0.0017 L1: 6.8647
Epoch [4/50] Batch [250/1600] LossG: 7.7641 LossD: 0.0014 L1: 6.7620
Epoch [4/50] Batch [300/1600] LossG: 7.6951 LossD: 0.0013 L1: 6.6935
Epoch [4/50] Batch [350/1600] LossG: 7.6830 LossD: 0.0020 L1: 6.6796
Epoch [4/50] Batch [400/1600] LossG: 7.6725 LossD: 0.0018 L1: 6.6696
Epoch [4/50] Batch [450/1600] LossG: 7.6444 LossD: 0.0017 L1: 6.6418
Epoch [4/50] Batch [500/1600] LossG: 7.6396 LossD: 0.0015 L1: 6.6374
Epoch [4/50] Batch [550/1600] LossG: 7.6312 LossD: 0.0016 L1: 6.6287
Epoch [4/50] Batch [600/1600] LossG: 7.6517 LossD: 0.0019 L1: 6.6490
Epoch [4/50] Batch [650/1600] LossG: 7.6460 LossD: 0.0028 L1: 6.6418
Epoch [4/50] Batch [700/1600] LossG: 7.6687 LossD: 0.0028 L1: 6.6646
Epoch [4/50] Batch [750/1600] LossG

...

C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 163 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)


Epoch [44/50] Batch [50/1600] LossG: 5.6254 LossD: 0.0000 L1: 4.6255
Epoch [44/50] Batch [100/1600] LossG: 5.7232 LossD: 0.0000 L1: 4.7233
Epoch [44/50] Batch [150/1600] LossG: 5.6856 LossD: 0.0000 L1: 4.6854
Epoch [44/50] Batch [200/1600] LossG: 5.7399 LossD: 0.0000 L1: 4.7397
Epoch [44/50] Batch [250/1600] LossG: 5.7637 LossD: 0.0000 L1: 4.7636
Epoch [44/50] Batch [300/1600] LossG: 5.7905 LossD: 0.0000 L1: 4.7903
Epoch [44/50] Batch [350/1600] LossG: 5.7746 LossD: 0.0000 L1: 4.7745
Epoch [44/50] Batch [400/1600] LossG: 5.7922 LossD: 0.0034 L1: 4.7979
Epoch [44/50] Batch [450/1600] LossG: 5.7989 LossD: 0.0031 L1: 4.8034
Epoch [44/50] Batch [500/1600] LossG: 5.7970 LossD: 0.0028 L1: 4.8009
Epoch [44/50] Batch [550/1600] LossG: 5.8048 LossD: 0.0026 L1: 4.8082
Epoch [44/50] Batch [600/1600] LossG: 5.7985 LossD: 0.0024 L1: 4.8017
Epoch [44/50] Batch [650/1600] LossG: 5.7960 LossD: 0.0022 L1: 4.7989
Epoch [44/50] Batch [700/1600] LossG: 5.8041 LossD: 0.0021 L1: 4.8069
Epoch [44/50] Batch [

C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 69 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)


Epoch [48/50] Batch [50/1600] LossG: 5.8653 LossD: 0.0001 L1: 4.8658
Epoch [48/50] Batch [100/1600] LossG: 5.8865 LossD: 0.0000 L1: 4.8867
Epoch [48/50] Batch [150/1600] LossG: 5.8416 LossD: 0.0000 L1: 4.8418
Epoch [48/50] Batch [200/1600] LossG: 5.8471 LossD: 0.0000 L1: 4.8472
Epoch [48/50] Batch [250/1600] LossG: 5.8485 LossD: 0.0000 L1: 4.8486
Epoch [48/50] Batch [300/1600] LossG: 5.8171 LossD: 0.0000 L1: 4.8172
Epoch [48/50] Batch [350/1600] LossG: 5.8111 LossD: 0.0000 L1: 4.8112
Epoch [48/50] Batch [400/1600] LossG: 5.8193 LossD: 0.0000 L1: 4.8194
Epoch [48/50] Batch [450/1600] LossG: 5.8224 LossD: 0.0000 L1: 4.8225
Epoch [48/50] Batch [500/1600] LossG: 5.8411 LossD: 0.0000 L1: 4.8409
Epoch [48/50] Batch [550/1600] LossG: 5.8331 LossD: 0.0000 L1: 4.8329
Epoch [48/50] Batch [600/1600] LossG: 5.8433 LossD: 0.0000 L1: 4.8431
Epoch [48/50] Batch [650/1600] LossG: 5.8313 LossD: 0.0000 L1: 4.8311
Epoch [48/50] Batch [700/1600] LossG: 5.8158 LossD: 0.0000 L1: 4.8156
Epoch [48/50] Batch [

C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 62 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)
C:\Users\len\AppData\Local\Temp\ipykernel_4696\1628243315.py:12: UserWarning: Conversion from CIE-LAB, via XYZ to sRGB color space resulted in 21 negative Z values that have been clipped to zero
  rgb = color.lab2rgb(lab)


Notes & Next steps
 - This notebook is a starting point. You should verify dataset pairing (names) and adjust the path logic in `PairedSARDataset` if needed.
 - Experiment with L1 weight, perceptual losses (VGG), and SSIM/LPIPS metrics for better visual fidelity.
 - For large images, use tiled inference with overlap and blending to remove seam artifacts.
 - If you have dual-polarization SAR (VV+VH), change `in_channels=2` in the UNet and modify the dataset to return two-channel inputs.
 - To fine-tune per-terrain, train the general model first then reinitialize the dataset to only that terrain and continue training from the saved best checkpoint.
